# Module 6 EDA Project - Coffee Quality Analysis Project

In this project, we explore the world of coffee by combining and cleaning two datasets scraped from the Coffee Quality Institute's website: **arabica_ratings_raw** and **robusta_ratings_raw**. Our goal is to perform exploratory data analysis (EDA) to answer various questions about the different types of coffees that have been scored throughout the years. 

Through this analysis, we aim to uncover insights related to coffee varieties, their quality ratings, and trends from country to country, ultimately enhancing our understanding of coffee quality and preferences.


In [1]:
import logging
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Set display options to show all rows and columns without truncation
# pd.set_option('display.max_rows', None)  # Show all rows
# pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.width', None)  # Adjust width to avoid truncation

In [2]:
# Load the datasets
arabica_data_raw = pd.read_csv('data/arabica_ratings_raw.csv')
robusta_data_raw = pd.read_csv('data/robusta_ratings_raw.csv')

In [ ]:
arabica_data_raw.head(1)

In [ ]:
# Drop columns where all values are NaN
arabica_data_drop_nan = arabica_data_raw.dropna(axis=1, how='all')
robusta_data_drop_nan = robusta_data_raw.dropna(axis=1, how='all')
arabica_data_drop_nan.head(3)
robusta_data_drop_nan.head(3)

In [ ]:
# Display the arabica_data_cleaned column headers
arabica_data_drop_nan.columns.tolist()

In [ ]:
# Display the robusta_data_cleaned column headers
robusta_data_drop_nan.columns.tolist()

In [ ]:
# Create a dictionary mapping the robusta column names to the arabica column names
robusta_column_mapping = {
    'Fragrance / Aroma': 'Aroma',
    'Salt / Acid': 'Acidity',  # Rename Salt / Acid to Acidity
    'Uniform Cup': 'Uniformity',
    'Bitter / Sweet': 'Sweetness'
}

# Rename the columns and add the 'Body' column in one step
robusta_data_drop_nan = (robusta_data_drop_nan
                         .rename(columns=robusta_column_mapping)
                         .assign(Body=np.nan))

robusta_data_drop_nan

In [8]:
# Specify which columns to keep in cleaned DataFrame
columns_to_keep = [
 'quality_score',
 'Species',
 'Owner',
 'Country of Origin',
 'Farm Name',
 'Lot Number',
 'Mill',
 'Company',
 'Altitude',
 'Region',
 'Producer',
 'Number of Bags',
 'Bag Weight',
 'In-Country Partner',
 'Harvest Year',
 'Grading Date',
 'Owner.1',
 'Variety',
 'Processing Method',
 'Aroma',
 'Flavor',
 'Aftertaste',
 'Acidity',
 'Body',
 'Balance',
 'Uniformity',
 'Clean Cup',
 'Sweetness',
 'Cupper Points',
 'Total Cup Points',
 'Moisture',
 'Category One Defects',
 'Quakers',
 'Color',
 'Category Two Defects',
 'Expiration',
 'Certification Body'
]

In [ ]:
# Select the columns from both DataFrames
arabica_selected = arabica_data_drop_nan[columns_to_keep]
robusta_selected = robusta_data_drop_nan[columns_to_keep]

# Concatenate the DataFrames
combined_coffee_data = pd.concat([arabica_selected, robusta_selected], axis=0, ignore_index=True)
combined_coffee_data.head(5)

In [ ]:
# Query the combined_coffee_data dataframe to show only the Robusta species rows
combined_coffee_data.query('Species == "Robusta"').head()

## Note:
While cleaning the column headers, I noticed there was a difference between the arabica and robusta columns. 
The arabica scores were:
* Aroma
* Flavor	
* Aftertaste	
* Acidity	
* Body	
* Balance		
* Uniformity	
* Clean Cup	
* Sweetness	
* Total Cupper Points

### and the Robusta scores were:
* Fragrance / Aroma	
* Flavor	
* Aftertaste	
* Salt / Acid	
* Bitter / Sweet	
* Mouthfeel	
* Uniform Cup	
* Clean Cup	
* Balance	
* Total Cupper Points

So I made the decision to rename some of the Robusta scores columns.

In [ ]:
# Specify the desired column order in a list
desired_order = [

    # Coffee Details
    'Species', 'Variety','Country of Origin', 'Region', 'Altitude', 'Processing Method', 'Harvest Year',
    
    # General Information
    'Farm Name', 'Lot Number',
    'Number of Bags', 'Producer', 'Mill',
    'Owner', 'Company', 'In-Country Partner',

    # Defects and Quality
    'Moisture', 'Category One Defects', 'Category Two Defects', 
    'Quakers', 'Color',

    # Cupping Scores
    'Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body', 'Balance', 
    'Uniformity', 'Clean Cup', 'Sweetness', 'Cupper Points', 'Total Cup Points',
    'Grading Date', 'Certification Body'
]

# Reorder the DataFrame columns
combined_coffee_data_reordered = combined_coffee_data[desired_order]

# Set pandas to display all columns
pd.set_option('display.max_columns', None)  # Set to None to show all columns

# Display the reordered DataFrame
combined_coffee_data_reordered

### Now we have a combined dataframe, combined_coffee_data_reordered, we can start to look at specific issues such as date formatting and missing values.

In [ ]:
# Use .loc to change all instances of NaN and 'Other' to 'Not Specified' in the Variety column
combined_coffee_data_reordered.loc[:, 'Variety'] = combined_coffee_data_reordered['Variety'].replace({
    np.nan: 'Not Specified',
    'Other': 'Not Specified'
})

# Print out all unique strings in the Variety column to check if there are any others that need to be altered
print(combined_coffee_data_reordered['Variety'].unique())

In [ ]:
def clean_total_cup_points(df, column_name):
    # Check if the column exists
    if column_name in df.columns:
        # Check the data type of the column
        if df[column_name].dtype == 'object':  # Ensure it is a string type before cleaning
            # Remove 'Sample' and any extra whitespace
            df.loc[:, column_name] = (
                df[column_name]
                .str.replace(r'Sample\s*', '', regex=True)  # Remove 'Sample' with optional trailing spaces
                .str.strip()  # Remove leading/trailing whitespace
            )
        # Convert to float, coercing errors to NaN
        df.loc[:, column_name] = pd.to_numeric(df[column_name], errors='coerce')

# Example usage
clean_total_cup_points(combined_coffee_data_reordered, 'Total Cup Points')

# Print the data type and updated scores to verify
print("Data type after conversion:")
print(combined_coffee_data_reordered['Total Cup Points'].dtype)
print("First 15 entries:")
print(combined_coffee_data_reordered['Total Cup Points'].head(15))

In [ ]:
# Print unique values before modification for debugging
#print(combined_coffee_data_reordered['Country of Origin'].unique())

# Define a mapping for the standardization
country_corrections = {
    'Cote d?Ivoire': "Côte d'Ivoire",
    'United States (Hawaii)': 'United States',
    'United States (Puerto Rico)': 'United States',
    'Tanzania, United Republic Of': 'United Republic of Tanzania'
    # Add more corrections as necessary
}

# Apply the corrections using .loc
combined_coffee_data_reordered.loc[:, 'Country of Origin'] = combined_coffee_data_reordered['Country of Origin'].replace(country_corrections)

# Replace NaN with 'Not Specified' using .loc
combined_coffee_data_reordered.loc[:, 'Country of Origin'] = combined_coffee_data_reordered['Country of Origin'].fillna('Not Specified')

# Verify the unique values after cleaning
print(combined_coffee_data_reordered['Country of Origin'].unique())


In [ ]:
# Create a new variable for cleaned data
cleaned_coffee_data = combined_coffee_data_reordered.copy()

# Create a 'Coffee ID' column
cleaned_coffee_data.loc[:, 'Coffee ID'] = cleaned_coffee_data.index + 1  # Start ID from 1

# Move 'Coffee ID' to the first column
cleaned_coffee_data = cleaned_coffee_data[['Coffee ID'] + [col for col in cleaned_coffee_data.columns if col != 'Coffee ID']]

# Optionally, check the result
cleaned_coffee_data

In [ ]:
cleaned_coffee_data.head(5)

### After we're done cleaning the data, let's perform some EDA.

In [ ]:
cleaned_coffee_data.describe()
# Describe them by country, by Processing Method, and Variety

In [ ]:
print(cleaned_coffee_data['Total Cup Points'].unique())

In [ ]:
# Assuming cleaned_coffee_data is your DataFrame
# Step 1: Count of Coffees by Country of Origin
country_counts = cleaned_coffee_data['Country of Origin'].value_counts().reset_index()
country_counts.columns = ['Country of Origin', 'Coffee Count']

# Step 2: Calculate Mean and Median Total Cup Points
mean_median_points = cleaned_coffee_data.groupby('Country of Origin')['Total Cup Points'].agg(['mean', 'median']).reset_index()
mean_median_points.columns = ['Country of Origin', 'Mean Total Cup Points', 'Median Total Cup Points']

# Step 3: Merge the DataFrames
country_summary = pd.merge(country_counts, mean_median_points, on='Country of Origin')

# Display the summary DataFrame
print(country_summary)

# Step 4: Box-and-Whisker Plot
plt.figure(figsize=(12, 6))
sns.boxplot(x='Country of Origin', y='Total Cup Points', data=cleaned_coffee_data)
plt.xticks(rotation=90)
plt.title('Box-and-Whisker Plot of Total Cup Points by Country of Origin')
plt.xlabel('Country of Origin')
plt.ylabel('Total Cup Points')
plt.tight_layout()
plt.show()


Questions to answer:
* Which varieties of coffee have the highest Total Cup Points?
* Which country has the highest top 5 scores, and which countries have the bottom 5 lowest scores?
* How do Arabica coffees compare to Robusta coffees in terms of Flavor, Sweetness, and Total Cupping Scores?

In [ ]:
# Which varieties of coffee have the highest Total Cup Points? 

# Calculate maximum scores for each variety
top_scores = (
    cleaned_coffee_data.groupby('Variety')
    .agg({'Total Cup Points': 'max'})
    .reset_index()
)

# Rename the columns for easier access
top_scores.columns = ['Variety', 'Top Score']

# Sort the DataFrame from highest to lowest top score and get the top 5
top_varieties = top_scores.sort_values(by='Top Score', ascending=False).head(5)

# Set the style of seaborn
sns.set_theme(style="whitegrid")

# Create a larger bar plot for top scores
plt.figure(figsize=(14, 8))  # Adjusted the figure size
bar_plot = sns.barplot(data=top_varieties, x='Variety', y='Top Score', palette='viridis')

# Set the y-axis limits
plt.ylim(80, 100)

plt.title('Top 5 Coffee Varieties by Highest Scores', fontsize=16)
plt.xlabel('Coffee Variety', fontsize=14)
plt.ylabel('Highest Total Cup Points', fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()  # Adjusts the plot to fit into the figure area.
plt.show()


In [ ]:
# Which country has the highest top 5 scores, and which countries have the bottom 5 lowest scores?

# Calculate maximum scores for each country
country_scores = (
    cleaned_coffee_data.groupby('Country of Origin')
    .agg({'Total Cup Points': 'max'})
    .reset_index()
)

# Rename the columns for easier access
country_scores.columns = ['Country', 'Top Score']

# Get the top 5 countries with highest scores
top_countries = country_scores.sort_values(by='Top Score', ascending=False).head(5)

# Get the bottom 5 countries with lowest scores
bottom_countries = country_scores.sort_values(by='Top Score').head(5)

# Display the results
print("Top 5 Countries by Highest Scores:")
print(top_countries)

print("\nBottom 5 Countries by Lowest Scores:")
print(bottom_countries)


In [ ]:
# How do Arabica coffees compare to Robusta coffees in terms of Flavor, Sweetness, and Total Cupping Scores?

# Set the style of seaborn
sns.set_theme(style="whitegrid")

# Create a figure with subplots
plt.figure(figsize=(18, 8))

# Set common y-axis limits (adjust these values based on your data range)
y_limits = (0, 10)  # Replace with appropriate min and max values for Flavor and Sweetness
cup_points_limits = (80, 100)  # Example limits for Total Cup Points

# Arabica Flavor Distribution
plt.subplot(2, 3, 1)  # 2 rows, 3 columns, 1st subplot
sns.boxplot(data=cleaned_coffee_data[cleaned_coffee_data['Species'] == 'Arabica'], y='Flavor', color='skyblue')
plt.title('Arabica Flavor Distribution')
plt.ylabel('Flavor Score')
plt.ylim(y_limits)  # Set y-axis limits for Flavor

# Arabica Sweetness Distribution
plt.subplot(2, 3, 2)  # 2 rows, 3 columns, 2nd subplot
sns.boxplot(data=cleaned_coffee_data[cleaned_coffee_data['Species'] == 'Arabica'], y='Sweetness', color='skyblue')
plt.title('Arabica Sweetness Distribution')
plt.ylabel('Sweetness Score')
plt.ylim(y_limits)  # Set y-axis limits for Sweetness

# Arabica Total Cup Points Distribution
plt.subplot(2, 3, 3)  # 2 rows, 3 columns, 3rd subplot
sns.boxplot(data=cleaned_coffee_data[cleaned_coffee_data['Species'] == 'Arabica'], y='Total Cup Points', color='skyblue')
plt.title('Arabica Total Cup Points Distribution')
plt.ylabel('Total Cup Points')
plt.ylim(cup_points_limits)  # Set y-axis limits for Total Cup Points

# Robusta Flavor Distribution
plt.subplot(2, 3, 4)  # 2 rows, 3 columns, 4th subplot
sns.boxplot(data=cleaned_coffee_data[cleaned_coffee_data['Species'] == 'Robusta'], y='Flavor', color='salmon')
plt.title('Robusta Flavor Distribution')
plt.ylabel('Flavor Score')
plt.ylim(y_limits)  # Set y-axis limits for Flavor

# Robusta Sweetness Distribution
plt.subplot(2, 3, 5)  # 2 rows, 3 columns, 5th subplot
sns.boxplot(data=cleaned_coffee_data[cleaned_coffee_data['Species'] == 'Robusta'], y='Sweetness', color='salmon')
plt.title('Robusta Sweetness Distribution')
plt.ylabel('Sweetness Score')
plt.ylim(y_limits)  # Set y-axis limits for Sweetness

# Robusta Total Cup Points Distribution
plt.subplot(2, 3, 6)  # 2 rows, 3 columns, 6th subplot
sns.boxplot(data=cleaned_coffee_data[cleaned_coffee_data['Species'] == 'Robusta'], y='Total Cup Points', color='salmon')
plt.title('Robusta Total Cup Points Distribution')
plt.ylabel('Total Cup Points')
plt.ylim(cup_points_limits)  # Set y-axis limits for Total Cup Points

plt.tight_layout()  # Adjusts the plot to fit into the figure area.
plt.show()
